In [36]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os
from dotenv import load_dotenv

In [37]:
load_dotenv()

llm= HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    huggingfacehub_api_token= os.getenv("HUGGINGFACE_API_ENDPOINT"),
    max_new_tokens= 500,
    temperature= 0.3
)

model= ChatHuggingFace(llm= llm)

In [38]:
class BlogState(TypedDict):
    outline: str
    title: str
    content: str
    evaluate: float

In [39]:
def create_outline(state: BlogState)-> BlogState:
    title= state['title']
    
    prompt= f"generate a outline for the blog on the topic: {title}"
    
    state['outline']= model.invoke(prompt).content
    
    return state

In [40]:
def create_blog(state: BlogState)-> BlogState:
    outline= state['outline']
    title= state['title']
    
    prompt= f"Write a detailed blog on the title: {title} using the given outline: {outline}"
    
    blog= model.invoke(prompt).content
    state['content']= blog
    
    return state

In [41]:
def evaluate_blog(state:BlogState):
    content= state['content']
    outline= state['outline']
    
    prompt= f"Based on the outline({outline}) can you rate the blog({content}) out of 10. Just provide the number in float datatype and nothing else"
    
    state['evaluate']= float(model.invoke(prompt).content)
    
    return state

In [42]:
graph= StateGraph(BlogState)

graph.add_node('create_outline', create_outline)
graph.add_node('create_blog', create_blog)
graph.add_node('evaluate_blog', evaluate_blog)

graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', 'evaluate_blog')
graph.add_edge('evaluate_blog', END)

workflow= graph.compile()

In [43]:
initial_state= {'title': 'Cars'}
final_state= workflow.invoke(initial_state)

print(final_state)

{'outline': 'Here\'s an outline for a blog on the topic "Cars":\n\n**I. Introduction**\n\n* Brief overview of the importance of cars in our lives\n* Explanation of the purpose of the blog post\n* Thesis statement: This blog post will explore the latest trends, features, and technologies in the car industry, as well as provide tips and advice for car buyers and enthusiasts.\n\n**II. The Evolution of Cars**\n\n* History of cars: from horse-drawn carriages to electric vehicles\n* Key milestones in car development: innovations, technologies, and design trends\n* Examples of iconic cars: their design, performance, and cultural impact\n\n**III. Latest Trends and Features in the Car Industry**\n\n* Electric and hybrid vehicles: benefits, challenges, and future prospects\n* Autonomous driving: current state, applications, and regulatory frameworks\n* Advanced safety features: airbags, lane departure warning systems, and more\n* Infotainment systems: integration with smartphones, voice assistan

In [44]:
print(final_state['evaluate'])

8.4


In [ ]:
workflow.